# The Transaction [class](https://docs.python.org/3/tutorial/classes.html)
## Attributes
A transaction is defined as an object with the following attributes:
1. Date
2. Total amount
3. Transaction type
4. Symbol (optional)
5. Number of shares (optional)
6. Description (optional)

[Function annotations](https://www.python.org/dev/peps/pep-3107/) capture the expected type of input for each of these attributes, and [default argument values](https://docs.python.org/3/tutorial/controlflow.html#default-argument-values) are used for the optional [keyword arguments](https://docs.python.org/3/tutorial/controlflow.html#keyword-arguments) per the below [`__init__()`](https://docs.python.org/3/reference/datamodel.html#object.__init__) syntax:

In [12]:
import datetime
from typing import Union

class Transaction:
    """Represents a bank transfer, buy, dividends, fee, or sell"""

    def __init__(self, date: datetime.date, total_amount: Union[float, int],
                 transact_type: str, symbol: str=None,
                 num_shares: int=None, description: str=None):
        """Creates object if valid parameters, else raises error"""
        try:  # Check input validity before assignment
            self._check_arg_types(date, total_amount, transact_type)
            self._check_arg_vals(total_amount, transact_type)
            self._check_kwarg_types(symbol, num_shares, description)
            self._check_kwarg_vals(transact_type, symbol, num_shares)
        except (AttributeError, ValueError) as e:
            raise e
        else:  # Create an object if data is valid
            self.date = date
            self.total_amount = float(total_amount)
            self.transact_type = transact_type
            self.symbol = symbol
            self.num_shares = num_shares
            self.description = description

Note that the [`Union`](https://docs.python.org/3/library/typing.html#typing.Union) facility specifies that `total_amount` is expected as either a `float` or as an `int` (although the value is later cast to a `float`)

If all validity checks are successful (do not raise any [Exceptions](https://docs.python.org/3/tutorial/errors.html)), then a Transaction [instance](https://docs.python.org/3/tutorial/classes.html#instance-objects) is created. While `total_amount` can be initialized with either an `int` or a `float` argument, it is cast to a `float` for consistency
## Validity checking

Input validity is checked before assignment to verify data type and values of both [positional](https://docs.python.org/3/glossary.html) and [keyword arguments](https://docs.python.org/3/tutorial/controlflow.html#keyword-arguments). If any of the [private methods](https://docs.python.org/3/tutorial/classes.html#private-variables), designated by `self._check_XXXX()`, raise an [exception](https://docs.python.org/3/library/exceptions.html), then the exception is re-raised and no object is created. If the `__init__()` [parameters](https://docs.python.org/3/faq/programming.html#faq-argument-vs-parameter) are valid, then an object is created

Checks against the data type of an argument can raise an `AttributeError`, and checks against the data contents of an argument can raise a `ValueError`
### `_check_arg_types()`
The first checker method, `_check_arg_types()`, checks the data type of the positional arguments `date`, `total_amount`, and `transact_type`:

In [8]:
class Transaction:
...
    def _check_arg_types(self, date, total_amount, transact_type):
        """Verify positional arguments are correct data type"""
        # Key is argument, value is tuple with expected format(s) and
        # associated error message
        checks = dict([(date, (datetime.date, "date type")),
                       (total_amount, ((float, int), "total_amount type")),
                       (transact_type, (str, "transact_type type"))])
        self._check_types(checks)
...

The method assembles a `checks` [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) with keys that correspond to the positional arguments from the `__init__()` method. The value for each key is a [tuple](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences) that contains:
1. The expected data type(s) for each argument
2. The error message to be raised is the argument type is wrong

The `checks` dictionary is then passed to `_check_types()`:
### `_check_types()`

In [9]:
class Transaction:
...
    def _check_types(self, checks, keyword_args=False):
        """Verify positional/keyword arguments are correct type"""
        # Key is argument, value is tuple with expected format(s) and
        # associated error message
        for arg, (arg_type, error_msg) in checks.items():
            # For positional arguments, just check if is correct type
            # For keyword args, check if is None or is correct type
            if ((not isinstance(arg, arg_type)) and
                    (not (keyword_args == True and arg is None))):
                raise AttributeError(error_msg)
...

The method accepts the `checks` dictionary described above and uses [looping technique](https://docs.python.org/3.4/tutorial/datastructures.html#looping-techniques) [unpacking syntax](https://docs.python.org/3.4/tutorial/controlflow.html#unpacking-argument-lists) to check if each argument is of the correct data type

Note the `keyword_args` keyword argument with a default value of `False` (which will be described later). For an arbitrary key in the `checks` dictionary, the first line of the `if` statement, using [`isinstance`](https://docs.python.org/3/library/functions.html#isinstance), reads as:

*if the argument is not of the specified type*

Since `_check_arg_types()` calls `_check_types()` with just one argument (the `checks` dictionary), `keyword_args` is `False` and the second line of the `if` statement evaluates to `True`

This means that if any of the positional arguments to `__init__()` do not have the data type specified in `checks`, then an `AttributeError` will be raised with the resultant message from the value tuple of the `checks` dictionary 
### `_check_kwarg_types()`

Keyword arguments in the `__init__()` method are checked via `_check_kwarg_types()`:

In [15]:
class Transaction:
...
    def _check_kwarg_types(self, symbol, num_shares, description):
            """Verify keyword arguments are correct data type"""
            # Key is argument, value is tuple with expected format(s) and
            # associated error message
            checks = dict([(symbol, (str, "symbol type")),
                           (num_shares, (int, "num_shares type")),
                           (description, (str, "description type"))])
            self._check_types(checks, keyword_args=True)
...

The structure of this function is identical to `_check_arg_types()`, except that the call to `_check_types()` sets the keyword argument `keyword_args` to `True`

`_check_types()` loops over the `checks` dictionary, but the second line of the `if` statement does not automatically evaluate to `True` as before:

In [ ]:
if ((not isinstance(arg, arg_type)) and
        (not (keyword_args == True and arg is None))):

Since `keyword_args` is always `True` when called by `_check_kwarg_types()`, the second line can be read as:

*if the argument is not `None`*

Thus the `if` statement will raise an `AttributeError` with the appropriate message for a keyword argument in `__init__()` if the argument is not of the type specified in the `checks` dictionary of `_check_kwarg_types()` and the argument is not `None`

### `_check_arg_vals()` and `ledger.transact_types`
As for the actual contents of the arguments, there are additional checks on arguments passed to `__init__()`. For example, the `total_amount` of a transaction is assumed to be a positive value as defined in `_check_arg_vals()`:

In [22]:
class Transaction:
...
    def _check_arg_vals(self, total_amount, transact_type):
        """Verify positional argument values are valid"""
        # Order-dependent list of checks and failure message to report
        # if condition is true
        checks = [(total_amount < 0, "total_amount negative"),
                  (transact_type not in transact_types, "transact_type type")]
        self._check_vals(checks)
...

In this private method checker function, the `checks` data structure is a list of tuples containing:
1. A condition that specifies invalid data
2. A message associated with the invalid data condition

Note that the `transact_type` must be in the `transact_types` [global variable](https://docs.python.org/3/tutorial/modules.html#more-on-modules) of ledger.py:

In [ ]:
# Contents of ledger.py
import datetime
from typing import Union

transact_types = set(['Bank transfer', 'Buy', 'Dividends', 'Fees', 'Sell'])

class Transaction:
...

Thus it is enforced that a transaction type is either `'Bank transfer'`, `'Buy'`, etc. The `_check_arg_vals()` method calls `_check_vals()`:

### `_check_vals()`

In [21]:
class Transaction:
...
    def _check_vals(self, checks):
        """Verify positional/keyword arguments have reasonable vals"""
        # Order-dependent list of checks and failure message to report
        # if condition is true
        for check, error_message in checks:
            if True == check:
                raise ValueError(error_message)
...

`_check_vals()` simply loops over the `checks` list and raises a `ValueError` with the resultant message as assigned in `_check_arg_vals()`

### `_check_kwarg_vals()`
`_check_kwarg_vals()` composes a `checks` list similar to that of `_check_arg_vals()` and calls `_check_vals()` in the same way:

In [23]:
class Transaction:
...
    def _check_kwarg_vals(self, transact_type, symbol, num_shares):
        """Check that keyword argument values are valid"""
        # Order-dependent list of checks and failure message to report
        # if condition is true
        checks = [
            (symbol is not None and (len(symbol) == 0 or symbol.isspace()),
                "blank symbol"),
            (transact_type == 'Bank transfer' and symbol is not None,
                "bank transfer has symbol"),
            (transact_type != 'Bank transfer' and symbol is None,
                "symbol missing"),
            (symbol is None and num_shares is not None,
                "num_shares should be None"),
            (symbol is not None and num_shares is None, "num_shares missing"),
            (num_shares is not None and num_shares <= 0,
                "non-positive num_shares")]
        self._check_vals(checks)
...

Note that there are more diverse checks in `_check_kwarg_vals()` than in `_check_arg_vals()`, explained as follows:
1. A symbol must have non-whitespace text if it is not `None`
2. A bank transfer can not have an associated symbol
3. A symbol must be associated with a transaction that is not a bank transfer
4. A transaction without a symbol can not have a number of shares
5. A transaction with a symbol must have a number of shares defined
6. If a transaction has a defined number of shares, the value must be positive